In [1]:
import pandas as pd
from collections import Counter
import numpy as np

# base_df = pd.read_csv("fake_data.csv", encoding_errors="ignore", on_bad_lines='skip', sep=",",)
base_df = pd.read_csv("BPI2016_Clicks_Logged_In.csv", encoding_errors="ignore", on_bad_lines='skip', sep=";",
                usecols=['CustomerID', 'AgeCategory', 'Gender', 'Office_U', 'Office_W',
       'SessionID', 'IPID', 'TIMESTAMP', 'VHOST', 'URL_FILE', 'PAGE_NAME',
       'REF_URL_category', 'page_load_error', 'page_action_detail', 'tip',
       'service_detail', 'xps_info'])
base_df.rename(columns={'URL_FILE':'Activity'}, inplace=True)
base_df["TIMESTAMP"] = pd.to_datetime(base_df["TIMESTAMP"], infer_datetime_format=True)
base_df = base_df.sort_values(["SessionID", "TIMESTAMP"])

C:\Users\krdeg\AppData\Local\Temp\ipykernel_10356\1976942568.py:6: DtypeWarning: Columns (11,13,14,15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  base_df = pd.read_csv("BPI2016_Clicks_Logged_In.csv", encoding_errors="ignore", on_bad_lines='skip', sep=";",


In [2]:
# Helper functions
def create_df_for_pivot(df: pd.DataFrame):
  df = df.copy()
  #Create consecutive column with the consecutive activity and count how many time each pair occurs
  df["Consecutive_1"] = df.groupby("SessionID")["Activity"].shift(periods=-1)

  #Count all the pairs and safe in new DataFrame
  df_for_pivot_1 = pd.DataFrame(Counter(list(zip(df['Activity'], df['Consecutive_1'],))), index=["value"]).T.reset_index()

  df_for_pivot_1.fillna("end_session", inplace=True)
  return df_for_pivot_1
  
def make_pivot(df, index_names, column_names):
  df = df.copy()
  """
  Functions that takes in a DataFrame and returns a pivot table with all the chances
  
  Create pivot table where chances are calculated that each row is succeded by activity that is represented in the column
  """
  df_chances = df.pivot_table(index=index_names, columns=column_names, values='value')
  
  #Calculate what the probabilities are by summing the row and dividing all the values in the row by total sum of the row
  df_chances["total_row_count"] = df_chances.sum(axis=1)
  df_chances = df_chances.div(df_chances["total_row_count"], axis=0)
  
  df_chances.fillna(0, inplace=True)
  df_chances.drop("total_row_count", axis=1, inplace=True)

  return df_chances

  
# Create df for pivot
df_for_pivot = create_df_for_pivot(base_df)
df_chances_1 = make_pivot(df_for_pivot, "level_0", "level_1")

#Calculate the probability of each activity that it is the first activity performed in the session
df_chances_1["start_session_chance"] = base_df.groupby("SessionID").nth(0)["Activity"].value_counts() / base_df.groupby("SessionID").nth(0)["Activity"].value_counts().sum()
df_chances_1.fillna(0, inplace=True)

df_chances_total = df_chances_1 
df_chances_total

#Split the start probabilities from the normal DataFrame
start_chances = df_chances_total["start_session_chance"][df_chances_total["start_session_chance"] > 0]

#Create df with all chances except the starting chance.
final_df = df_chances_total.iloc[:, :-1]
final_df

level_1,/,//,//portal/page/portal/werk_nl/werknemer/solliciteren/solliciteren-werk-zoeken/sollicitatiebrief,//werk_nl/werknemer/contact,//werk_nl/werknemer/contact/email_uw_vraag,//werk_nl/werknemer/contact/vestiging-zoeken,//werk_nl/werknemer/home,//werk_nl/werknemer/over-werk-nl/handleiding,//werk_nl/werknemer/over-werk-nl/handleiding/cv-plaatsen,//werk_nl/werknemer/over-werk-nl/handleiding/sollicitatie-wijziging,...,/xpsitem/wdo_013389,/xpsitem/wdo_013407,/xpsitem/wdo_013827,/xpsitem/wdo_014041,/zoeken_portlet/ajax/addVacatureToGereageerd,/zoeken_portlet/ajax/zoekAantalIndicatief,/zoeken_portlet/ajax/zoekBeroep,/zoeken_portlet_wg/ajax/selectContactpersoon,/zoeken_portlet_wg/ajax/selectVacature,end_session
level_0,,,,,,,,,,,,,,,,,,,,,
/,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.073834
//,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.035714
//portal/page/portal/werk_nl/werknemer/solliciteren/solliciteren-werk-zoeken/sollicitatiebrief,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000
//werk_nl/werknemer/contact,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000
//werk_nl/werknemer/contact/email_uw_vraag,0.0,0.0,0.0,0.0,0.029412,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.058824
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
/zoeken_portlet/ajax/addVacatureToGereageerd,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.017588,0.002513,0.005025,0.0,0.0,0.055276
/zoeken_portlet/ajax/zoekAantalIndicatief,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000107,0.537184,0.051269,0.0,0.0,0.007539
/zoeken_portlet/ajax/zoekBeroep,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.441592,0.489910,0.0,0.0,0.003979


In [3]:
def skip_event(_df:pd.DataFrame,event:str,deviation_rate):
  df = _df.copy()
  # print(f"Decrease the probability to reach event: {event} with deviation rate: {deviation_rate}")
  old_probability = df.loc[:,event]
  deviation_rate = deviation_rate / 100
  
  # decrease the probability
  for i,prob in old_probability.items():
    # skip row of event
    if i == event: continue
    
    # start changing the prob 
    if prob > 0:
      decrease = prob * (1-deviation_rate)
      df.loc[i,event] -= prob - decrease      
      for _i,_prob in df.loc[i,:].items():
        df.loc[i,_i] += (prob - decrease) * df.loc[event,_i]
  
  
  # change row of event
  if df.loc[event,event] > 0:
    decrease = df.loc[event,event] * (1 - deviation_rate)
    original_value = df.loc[event,event]
    count = df.loc[event]
    count = count[count > 0].__len__() - 1
    
    for i,prob in  df.loc[event,:].items():
      if df.loc[event,i] > 0:
        # Circle case
        if i == event: 
          df.loc[event,i] -= prob - decrease
        else: df.loc[event,i] += (original_value - decrease) / count 
      
  return df

In [38]:
def skip_event_2(_df:pd.DataFrame,deviation_rate:int,_list:list):
  df = _df.copy()
  for item in _list:        
        # lower the probability to reach eventB from eventA. 
        # And with that amount
        # increase the probability to reach eventC from eventA
        deviation_rate = deviation_rate / 100
        old_probability = _df.loc[item[0],item[1]]
        new_val = old_probability * (1 - deviation_rate)
        _df.loc[item[0],item[1]] = new_val
        _df.loc[item[0],item[2]] += old_probability - new_val
  return df

In [4]:
def skip_multiple_events(_list:list,prop_matrix,deviation_rate):
  res = []
  for item in _list:
    new_df = skip_event(prop_matrix,item[0],item[1])
    res.append({"new_df":new_df,
                "activity": item[0],
                
                "dev":deviation_rate})
  return res
    

In [15]:
def skip_mul_2(_list:list,prop_matrix,deviation_rate):
  res = []
  for item in _list:
    new_df = skip_event(prop_matrix,item[1],deviation_rate)
    res.append({"new_df":new_df,
                "activity": item[1],
                'a': item[0],
                'b': item[2],
                "dev":deviation_rate})
  return res
  

In [37]:
gen_patterns = pd.read_csv('gen_patterns/skipping/skipping_patterns_008_5.csv')
# transform to list
_list = gen_patterns.values.tolist()
gen_patterns


,a,b,c,a-b,b-c,a-c,a-b-c
0,/ajax/dwr/call/plaincall/JAddressFinder.getAdd...,/werk_nl/werknemer/mijn_werkmap/doorgeven/wijz...,end_session,0.666667,0.260685,0.007937,0.173790
1,/werk_nl/werknemer/mijn_werkmap/mijn-weg-naar-...,/werk_nl/werknemer/mijn_werkmap/mijn-weg-naar-...,/werk_nl/werknemer/mijn_werkmap/inschrijving/m...,0.255814,0.429825,0.003322,0.109955
2,/werk_nl/werknemer/mijn_werkmap/mijn-weg-naar-...,/werk_nl/werknemer/mijn_werkmap/mijn-weg-naar-...,/werk_nl/werknemer/mijn_werkmap/werk-zoeken/mi...,0.265781,0.352436,0.009967,0.093671
3,/werk_nl/werknemer/solliciteren/ontwikkel-uzelf,/portal/page/portal/werk_nl/werknemer/sollicit...,/werk_nl/werknemer/solliciteren/ontwikkel-uzel...,0.221088,0.350685,0.001701,0.077532
4,/werk_nl/werknemer/uitkering-aanvragen,/portal/page/portal/werk_nl/werknemer/uitkerin...,/portal/page/portal/home/diensten/aanvragen-ww,0.171661,0.464428,0.004622,0.079724
...,...,...,...,...,...,...,...
100,/werk_nl/werknemer/mijn_werkmap/doorgeven/mijn...,/werk_nl/werknemer/home,/werk_nl/werknemer/solliciteren/ontwikkel-uzel...,0.091857,0.000592,0.000005,0.000054
101,/werk_nl/werknemer/mijn_werkmap/postvak/mijn_d...,/werk_nl/werknemer/home,/werk_nl/werknemer/over-werk-nl/handleiding/va...,0.082469,0.000659,0.000007,0.000054
102,/werk_nl/werknemer/mijn_werkmap/postvak/mijn_d...,/werk_nl/werknemer/mijn_werkmap/postvak/mijn_b...,/werk_nl/werknemer/mijn_werkmap/inschrijving/m...,0.151873,0.000345,0.000007,0.000052
103,/werk_nl/werknemer/mijn_werkmap/postvak/mijn_d...,/werk_nl/werknemer/mijn_werkmap/postvak/mijn_b...,//werk_nl/werknemer/solliciteren/ontwikkel-uze...,0.151873,0.000276,0.000007,0.000042


In [44]:
skipped = skip_event_2(final_df,99,_list)

C:\Users\krdeg\AppData\Local\Temp\ipykernel_10356\2829916453.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  _df.loc[item[0],item[1]] = new_val
C:\Users\krdeg\AppData\Local\Temp\ipykernel_10356\2829916453.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  _df.loc[item[0],item[2]] += old_probability - new_val


In [55]:
import random


def generate_markov_chain_new(amount:int,start_chances,df_dict:pd.DataFrame, patters_df:pd.DataFrame) -> list:
  failed = 0
  generated_sessions = []
  while len(generated_sessions) < amount:
    flag = False
    new_session = []
    # choose start activity
    start_activity = np.random.choice(a = np.array(start_chances.index), size = 1, p = np.array(start_chances.values))
    # start_activity = np.random.choice(a = np.array(df_dict.iloc[0,:].index), size = 1, p = np.array(df_dict.iloc[0,:].values))
    new_session.append(start_activity[0])

    
    while new_session[-1] != 'end_session':
    # while new_session[-1] != 'E':
      curr_activity = new_session[-1]
      
      # check if current activity is in patterns_df column a
      check = patters_df[patters_df['a'] == curr_activity]
      if len(check) > 0:
        # next activity is in patterns_df column c where a is current activity
        next_activity = patters_df[patters_df['a'] == curr_activity]['c']
        new_session.append(random.choice(next_activity.values)) # this has te be improved
        flag = True
      else:
        row_activity = df_dict.loc[curr_activity,:]
        activity = np.random.choice(a = list(row_activity.keys()), size = 1, p = list(row_activity.values))
        new_session.append(activity[0])

    
    if flag: 
      generated_sessions.append(new_session) 
    else:
      failed += 1
      print({"failed":failed,"total":len(generated_sessions)}) 

    
  
  return generated_sessions

In [33]:
def generate_markov_chain(amount:int,start_chances,df_dict:pd.DataFrame, act1:str,act2:str) -> list:
  failed = 0
  generated_sessions = []
  while len(generated_sessions) < amount:
    flag = False
    new_session = []
    # choose start activity
    start_activity = np.random.choice(a = np.array(start_chances.index), size = 1, p = np.array(start_chances.values))
    # start_activity = np.random.choice(a = np.array(df_dict.iloc[0,:].index), size = 1, p = np.array(df_dict.iloc[0,:].values))
    new_session.append(start_activity[0])

    
    while new_session[-1] != 'end_session':
    # while new_session[-1] != 'E':
      curr_activity = new_session[-1]
      row_activity = df_dict.loc[curr_activity,:]
      activity = np.random.choice(a = list(row_activity.keys()), size = 1, p = list(row_activity.values))
      new_session.append(activity[0])
      if activity[0] == act1: print("YEAH")
      if flag == False:
        if new_session[-2] == act1 and new_session[-1] == act2:
          print("sesssss")
          flag = True
    
    if flag: 
      generated_sessions.append(new_session) 
    else:
      failed += 1 

    
  
  return generated_sessions

In [34]:
def generate_sessions(skipping_list:list ,total_amount:int):
  session_per_activity = total_amount / len(skipping_list)
  sessions = []
  for item in skipping_list:
    markov = generate_markov_chain(session_per_activity,start_chances,item["new_df"],item["a"],item["b"])
    sessions += markov
  return sessions
  

# markov_result = generate_sessions(skipped,100)
# res
# print(len(markov_result))

In [56]:
# markov_result = generate_sessions(skipped,100)
markov_res = generate_markov_chain_new(100,start_chances,skipped,gen_patterns)
# res
print(len(markov_res))

{'failed': 1, 'total': 7}
{'failed': 2, 'total': 8}
{'failed': 3, 'total': 9}
{'failed': 4, 'total': 12}
{'failed': 5, 'total': 15}
{'failed': 6, 'total': 18}
{'failed': 7, 'total': 19}
{'failed': 8, 'total': 42}
{'failed': 9, 'total': 46}
{'failed': 10, 'total': 53}
{'failed': 11, 'total': 53}
{'failed': 12, 'total': 53}
{'failed': 13, 'total': 62}
{'failed': 14, 'total': 65}
{'failed': 15, 'total': 65}
{'failed': 16, 'total': 76}
{'failed': 17, 'total': 79}
{'failed': 18, 'total': 80}
{'failed': 19, 'total': 82}
{'failed': 20, 'total': 85}
{'failed': 21, 'total': 98}
100


In [ ]:
def markov_result_to_df(list_markov_result, name_index, name_csv):
    activity_list_final = []
    session_id_list = []
    for nbr in range(len(markov_result)):
        string_generated = name_index + str(nbr)
        for i in markov_result[nbr]:
            if i != "end_session":
                activity_list_final.append(i)
                session_id_list.append(string_generated)
                
    df_generated = pd.DataFrame(list(zip(session_id_list, activity_list_final)),
               columns =['SessionID', 'URL_FILE'])
    df_generated.to_csv(name_csv)
    
markov_result_to_df(markov_result, "generated_top3_50%_", "gen_sessions/skipped/generated_withCircle_top3_50%_2500.csv")